<a href="https://colab.research.google.com/github/rahul-727/SPR-/blob/main/2348544_Lab8_SPR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

The Viterbi Algorithm ensures the speech recognition system correctly maps acoustic feature vectors (observations) to the phoneme sequence "hello," determining the most likely interpretation of the audio input.

* designed to help a computer recognize the word "hello" from a sequence of sounds or acoustic signals.
* Breaking Down the Word:
The word "hello" is split into smaller sound units called phonemes: /h/, /e/, /l/, /o/.
* Understanding the Sounds:
Each phoneme produces certain sound patterns, which the program identifies as observations (O1, O2, O3, O4).
* How Sounds are Linked:
The program uses rules (probabilities) to understand:How likely it is to move from one sound to the next (e.g., from /h/ to /e/), and
How likely a sound matches the observed signal (e.g., /h/ produces O1).
* When the program hears a sequence of sounds, it tries to figure out which sequence of phonemes (hidden states) is most likely responsible for those sounds. It calculates this using the Viterbi Algorithm, which finds the best path through a series of possible states.

Transition Probability Matrix (A)

* This defines the probability of transitioning from one hidden state (phoneme) to another.
Example: The probability of moving from /h/ to /e/ is 0.7.

Emission Probability Matrix (B)
* This defines the probability of emitting an observation (acoustic feature) given a specific hidden state.
Example: The probability of observing feature vector O1 from state /h/ is 0.6.

Initial Probabilities (π)

* This specifies the probability of starting in a particular hidden state.
Example: The probability of starting in state /h/ is 1.0.

Observation Sequence (O)
* This is the sequence of observed feature vectors [O1, O2, O3, O4], represented as indices [0, 1, 2, 3].

In [2]:
def viterbi_algorithm(A, B, pi, O):
    n_states = A.shape[0]
    n_observations = len(O)

    # Initialize the Viterbi table and backpointer table
    viterbi = np.zeros((n_states, n_observations))
    backpointer = np.zeros((n_states, n_observations), dtype=int)

    # Initialization step
    viterbi[:, 0] = pi * B[:, O[0]]

    # Recursion step
    for t in range(1, n_observations):
        for s in range(n_states):
            prob = viterbi[:, t-1] * A[:, s] * B[s, O[t]]
            viterbi[s, t] = np.max(prob)
            backpointer[s, t] = np.argmax(prob)

    # Termination step
    best_path_prob = np.max(viterbi[:, -1])
    best_last_state = np.argmax(viterbi[:, -1])

    # Backtrace to find the best path
    best_path = [best_last_state]
    for t in range(n_observations - 1, 0, -1):
        best_last_state = backpointer[best_last_state, t]
        best_path.insert(0, best_last_state)

    return best_path, best_path_prob

In [3]:
states = ["/h/", "/e/", "/l/", "/o/"]
observations = ["O1", "O2", "O3", "O4"]

A = np.array([
    [0.0, 0.7, 0.3, 0.0],
    [0.0, 0.2, 0.6, 0.2],
    [0.0, 0.0, 0.3, 0.7],
    [0.0, 0.0, 0.1, 0.9]
])

B = np.array([
    [0.6, 0.2, 0.1, 0.1],
    [0.1, 0.7, 0.1, 0.1],
    [0.1, 0.1, 0.6, 0.2],
    [0.2, 0.1, 0.2, 0.5]
])

pi = np.array([1.0, 0.0, 0.0, 0.0])

O = [0, 1, 2, 3]

In [4]:
best_path, best_path_prob = viterbi_algorithm(A, B, pi, O)
best_path_states = [states[state] for state in best_path]

print("Most likely sequence of phoneme states:", best_path_states)
print("Probability of the most likely sequence:", best_path_prob)

Most likely sequence of phoneme states: ['/h/', '/e/', '/l/', '/o/']
Probability of the most likely sequence: 0.03704399999999999


* The sequence of states ['/h/', '/e/', '/l/', '/o/'] represents the phonemes of the word "hello." This indicates that the observation sequence [O1, O2, O3, O4] corresponds to the acoustic features of these phonemes in the given order. Thus, the Viterbi Algorithm successfully maps the observations to the word "hello."

Probability of the Most Likely Sequence:
The computed probability of 0.03704 represents the likelihood of observing the given sequence of acoustic features [O1, O2, O3, O4] under the Hidden Markov Model (HMM) parameters (transition probabilities, emission probabilities, and initial probabilities).

* Although the probability is relatively small, this is expected in HMMs due to the multiplicative nature of probabilities, especially over multiple time steps.
This is the most likely path compared to all other possible paths through the HMM, meaning it is the best explanation for the observations.